In [1]:
# autoRMZ test/development notebook (v1).

In [2]:
# List all files to ensure all dependencies exist.
!ls

LICENSE    dataset    processor  testing.py
README.md  pipeline   setup.py   v1.ipynb


In [10]:
# Import dependencies
import cv2
import json
import keras
from pathlib import Path
import pandas as pd

In [2]:
# Create data paths
preprocessed_path = Path('./dataset/images')
processed_path = Path('./dataset/processed')
roi_path = Path('./dataset/roi')

In [3]:
# Get files from data paths
preprocessed_images = [e for e in preprocessed_path.iterdir() if e.is_file()]
processed_json = [e for e in processed_path.iterdir() if e.is_file()]

In [24]:
# Obtain ROI from images and save to csv
data = []

for json_file in processed_json:
    data_point_name = json_file.name[:-5]
    image_file = Path('./dataset/images/{}.png'.format(data_point_name))
    crop_areas = []
    with json_file.open() as f:
        crop_areas = json.load(f)['areas']
    
    for j in range(0, len(crop_areas)):
        area = crop_areas[j]
        x1 = min(area['p1'][0], area['p2'][0])
        y1 = min(area['p1'][1], area['p2'][1])
        x2 = x1 + abs(area['p1'][0] - area['p2'][0])
        y2 = y1 + abs(area['p1'][1] - area['p2'][1])
        data.append([
            image_file.absolute(),
            x1,
            y1,
            x2,
            y2,
            'meteor'
        ])

df = pd.DataFrame(data, columns=['path', 'x1', 'y1', 'x2', 'y2', 'object'])
# Save as csv
df.to_csv('./dataset/roi.csv')
# Save as annotated text file for training step
df.to_csv('./dataset/annotate.txt', header=None, index=None, sep=' ')